# Homework 12

https://scikit-learn.org/0.15/modules/scaling_strategies.html#incremental-learning

* Implement a mini batch functionality to train a regressor.
    - (Optional) If anyone want to do this in a pipeline can do this: https://koaning.github.io/tokenwiser/api/pipeline.html

* Save model, load the model again and test it on `X_test` __Do NOT commit the pickle file__

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
def test_df():
    df = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/car_prices/car_prices.csv', low_memory=False)

    df = df.sample(5000, random_state=100).reset_index(drop=True)
    
    y = df['sellingprice']
    df.drop('sellingprice', axis=1, inplace=True)
    X = df
    
    return X,y

def partial_df():
    df = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/car_prices/car_prices.csv', low_memory=False)
   
    while(True):
        yield df.sample(100).reset_index(drop=True)
        
gen = partial_df()

In [3]:
X_test, y_test = test_df()

In [4]:
gen

<generator object partial_df at 0x7fda5a7fd5b0>

In [5]:
# each time you call this you will get a new slice of the dataframe.
next(gen)

,year,make,model,trim,body,transmission,vin,state,condition,odometer,color,interior,seller,mmr,sellingprice,saledate
0,2014,Toyota,4Runner,Limited,SUV,NaN,jtebu5jr3e5177509,pa,1.9,11749.0,red,black,remarketing by ge/manheim pennsylvania,38900,34000,Fri Jun 12 2015 02:00:00 GMT-0700 (PDT)
1,2010,Chevrolet,Malibu,LT1,Sedan,automatic,1g1zc5e04af205705,wi,2.9,75301.0,white,black,"enterprise fleet management exchange, inc.",8025,8700,Wed Feb 18 2015 02:00:00 GMT-0800 (PST)
2,2013,Nissan,Frontier,SV,Crew Cab,automatic,1n6ad0er1dn744907,ga,4,36626.0,black,gray,remarketing by ge/the credit union loan source,18000,18000,Tue Jan 20 2015 05:10:00 GMT-0800 (PST)
3,2012,Mazda,CX-9,Touring,SUV,automatic,jm3tb3cv7c0334117,pa,4.6,26407.0,black,black,"jpmorgan chase bank,n.a.",22300,22000,Fri Jan 09 2015 09:00:00 GMT-0800 (PST)
4,2002,Chevrolet,Blazer,LS,SUV,automatic,1gncs18wx2k125544,oh,1,133358.0,silver,gray,halleen kia,1275,1400,Tue Jan 13 2015 09:30:00 GMT-0800 (PST)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,2013,Volkswagen,Beetle Convertible,Turbo PZEV,Beetle Convertible,automatic,3vw7t7at0dm833577,fl,4.5,10746.0,—,black,vw credit,20100,20000,Tue Jan 20 2015 01:30:00 GMT-0800 (PST)
96,2014,Hyundai,Accent,GLS,sedan,automatic,kmhct4ae0eu650738,il,1.9,26379.0,gray,gray,kfl llc;embarc llc,10400,7400,Tue Jun 16 2015 04:00:00 GMT-0700 (PDT)
97,2012,Toyota,RAV4,Base,SUV,automatic,2t3bf4dv5cw206712,oh,4.5,25472.0,silver,gray,toyota financial services,15250,16400,Mon Jun 01 2015 09:00:00 GMT-0700 (PDT)
98,2005,Dodge,Ram Pickup 2500,SLT,Quad Cab,automatic,3d7ks28d35g819549,pa,3.5,111519.0,black,black,mike watchers used cars,11850,13700,Fri Mar 06 2015 01:30:00 GMT-0800 (PST)


In [6]:
len(X_test)

5000

## Data preprocessing

In [7]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.compose import make_column_selector as selector
from sklearn.linear_model import SGDRegressor
import warnings
warnings.filterwarnings("ignore")

In [8]:
df_numerical_features = X_test.select_dtypes(exclude='object')
df_categorical_features = X_test.select_dtypes(include='object')

In [9]:
numerical_columns_selector = selector(dtype_exclude=object)
cat_columns_selector = selector(dtype_include =object)
cat_columns = cat_columns_selector(X_test)
numeric_columns =numerical_columns_selector(X_test)
print("Categorical features are ", cat_columns)
print("Numerial features are", numeric_columns)

Categorical features are  ['make', 'model', 'trim', 'body', 'transmission', 'vin', 'state', 'condition', 'color', 'interior', 'seller', 'mmr', 'saledate']
Numerial features are ['year', 'odometer']


In [10]:
numeric_pipeline = Pipeline(steps = [('imputation', SimpleImputer(missing_values = np.nan, strategy = 'median')),('scaler', StandardScaler())])
categoric_pipeline = Pipeline(steps = [('imputation', SimpleImputer(missing_values = np.nan, strategy = 'most_frequent')),('onehotencoder', OneHotEncoder(handle_unknown = 'ignore', drop = 'first'))])
preprocessor = ColumnTransformer(transformers = [('num', numeric_pipeline,numeric_columns),('cat',categoric_pipeline, cat_columns)])
preprocessor.fit(X_test)

ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('imputation',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 ['year', 'odometer']),
                                ('cat',
                                 Pipeline(steps=[('imputation',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('onehotencoder',
                                                  OneHotEncoder(drop='first',
                                                                handle_unknown='ignore'))]),
                                 ['make', 'model', 'trim', 'body',
                                  'transmission', 'vin', 'state', 'condition',
                                  'color', 'interior', 'seller', 'mmr',
                                  'saledate'])])

In [11]:
df = pd.read_csv('https://raw.githubusercontent.com/msaricaumbc/DS_data/master/ds602/car_prices/car_prices.csv', low_memory=False)
len(df)

558837

## Model Building 

In [12]:
from sklearn.linear_model import SGDRegressor
sgd = SGDRegressor(loss='squared_error', random_state=40)

In [13]:
!pip install pyprind

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


- Each time we Run next(gen) ,it gives 100 records. Now since data size is 558837. so I should run next(gen) for 5588 times. Thats why I chose total_batches as 100 and ran it for 56 epochs.

In [24]:
import pyprind
pbar = pyprind.ProgBar(56)
batches = 100
classes = np.array([0, 1])
datasize = 0
for _ in range(56):
    for batch in range(batches):
        batch_df = next(gen)
        X_train = batch_df.drop('sellingprice', axis=1)
        Y_train = batch_df['sellingprice']
        X_train = preprocessor.transform(X_train)
        sgd.partial_fit(X_train, Y_train)
        datasize += 100
    pbar.update()


0% [##############################] 100% | ETA: 00:00:00
Total time elapsed: 00:03:49


### Model Saving and Loading

In [ ]:
import joblib
joblib.dump(sgd, 'sgd_model.joblib') # Dump the model

sgd1 = joblib.load('sgd_model.joblib')  #Load the model

### Prediction

In [23]:
X_test_preprocessed = preprocessor.transform(X_test) # Preprocess the test data
y_pred = sgd.predict(X_test_preprocessed) # Test the model on X_test
sgd1.score(X_test_preprocessed,y_test)

0.8111888663116442

In [26]:
y_pred

array([13380.42847704, 19560.33399956, 14942.72754148, ...,
       21362.89280219, 19617.52173104, 16510.06669375])

- The SGD regressor model achieved decent accuracy of 81% . It took about 3 minutes to run using Partial_fit() feature which reduces memory usage compared to retraining the model on the entire dataset.It only needs to process new data points or small batches, resulting in faster training times.